<a href="https://colab.research.google.com/github/AiPals/My_Ai_Model/blob/main/EthicsTrainingBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#import tarfile

# Path to the .tar file and where to extract it
#tar_file_path = "/content/drive/MyDrive/ethics.tar"  # Replace with your .tar file path
#extract_path = "/content/drive/MyDrive/extracted_ethics_dataset/ethics"  # Replace with the folder where you want to extract

# Extract the .tar file
#with tarfile.open(tar_file_path, 'r') as file:
    #file.extractall(path=extract_path)


Mounted at /content/drive


In [ ]:
# Install required packages
!pip install transformers
!pip install torchtext

# Import libraries
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import get_linear_schedule_with_warmup
import torch.optim as optim
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load the dataset from Google Drive (Assuming it's a CSV file)
import pandas as pd
dataset_path = "/content/drive/MyDrive/extracted_ethics_dataset/ethics"  # Replace with your dataset path
df_justice_train = pd.read_csv("/content/drive/MyDrive/extracted_ethics_dataset/ethics/justice/justice_train.csv")
df_justice_test = pd.read_csv("/content/drive/MyDrive/extracted_ethics_dataset/ethics/justice/justice_test.csv")
df_justice_test_hard = pd.read_csv("/content/drive/MyDrive/extracted_ethics_dataset/ethics/justice/justice_test_hard.csv")
df_deontology_train = pd.read_csv("/content/drive/MyDrive/extracted_ethics_dataset/ethics/deontology/deontology_train.csv")
df_deontology_test = pd.read_csv("/content/drive/MyDrive/extracted_ethics_dataset/ethics/deontology/deontology_test.csv")
df_deontology_test_hard = pd.read_csv("/content/drive/MyDrive/extracted_ethics_dataset/ethics/deontology/deontology_test_hard.csv")
df_utilitarianism_train = pd.read_csv("/content/drive/MyDrive/extracted_ethics_dataset/ethics/utilitarianism/util_train.csv")
df_utilitarianism_test = pd.read_csv("/content/drive/MyDrive/extracted_ethics_dataset/ethics/utilitarianism/util_test.csv")
df_utilitarianism_test_hard = pd.read_csv("/content/drive/MyDrive/extracted_ethics_dataset/ethics/utilitarianism/util_test_hard.csv")
df_commonsense_train = pd.read_csv("/content/drive/MyDrive/extracted_ethics_dataset/ethics/commonsense/cm_train.csv")
df_commonsense_test = pd.read_csv("/content/drive/MyDrive/extracted_ethics_dataset/ethics/commonsense/cm_test.csv")
df_commonsense_test_hard = pd.read_csv("/content/drive/MyDrive/extracted_ethics_dataset/ethics/commonsense/cm_test_hard.csv")
df_commonsense_ambig = pd.read_csv("/content/drive/MyDrive/extracted_ethics_dataset/ethics/commonsense/cm_ambig.csv")
df_virtue_train = pd.read_csv("/content/drive/MyDrive/extracted_ethics_dataset/ethics/virtue/virtue_train.csv")
df_virtue_test = pd.read_csv("/content/drive/MyDrive/extracted_ethics_dataset/ethics/virtue/virtue_test.csv")
df_virtue_test_hard = pd.read_csv("/content/drive/MyDrive/extracted_ethics_dataset/ethics/virtue/virtue_test_hard.csv")
#print(df_deontology_train.columns)

# Preprocess the dataset
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
input_ids = []
attention_masks = []

for sentence in df_deontology_train['scenario']:
    # Your code here
    # Assuming the text column in your CSV is named 'sentence'
    encoded_dict = tokenizer.encode_plus(
    sentence,
    add_special_tokens=True,
    max_length=64,
    truncation=True,  # Explicitly truncate
    padding='max_length',  # Use new padding argument
    return_attention_mask=True,
    return_tensors='pt',
)

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(df_deontology_train['label'].values)

#labels = torch.tensor(df['label'].values)  # Assuming the label column in your CSV is named 'label'

# Create DataLoader
batch_size = 32
dataset = TensorDataset(input_ids, attention_masks, labels)
dataloader = DataLoader(dataset, batch_size=batch_size)

# Initialize BERT model
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,  # Update this based on the number of labels in your dataset
    output_attentions=False,
    output_hidden_states=False,
)
# Initialize your model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

# Move the model to GPU
model = model.to(device)



# Set up optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataloader))

# Fine-tuning loop
for epoch in range(3):  # Number of epochs
    for batch in dataloader:
        b_input_ids, b_attention_mask, b_labels = batch

        # Send the batch to GPU
        b_input_ids = b_input_ids.to(device)
        b_attention_mask = b_attention_mask.to(device)
        b_labels = b_labels.to(device)

        model.zero_grad()
        outputs = model(b_input_ids, attention_mask=b_attention_mask, labels=b_labels)
        loss = outputs[0]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

print("Fine-tuning completed.")
#for epoch in range(3):  # Number of epochs
    #for batch in dataloader:
        #b_input_ids, b_attention_mask, b_labels = batch
        #model.zero_grad()
        #outputs = model(b_input_ids, attention_mask=b_attention_mask, labels=b_labels)
        #loss = outputs[0]
        #loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        #optimizer.step()
        #scheduler.step()

#print("Fine-tuning completed.")


In [ ]:
!pip install transformers
from transformers import BertForSequenceClassification

# Load your model (this is just an example; your loading code might differ)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

# Then save it
model.save_pretrained("/content/drive/MyDrive/model/BERT_Train_Ethics_101")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.7 MB/s eta 0:00:00


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
